<a href="https://colab.research.google.com/github/DonghaeSuh/PyTorch_Basic/blob/main/train_valid_test_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 현재 위치 설정

In [1]:
cd drive/MyDrive/pytorch/

/content/drive/MyDrive/pytorch


#모듈 불러오기

In [2]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split

일단 Dataset과 모델 class를 만든 다음에\
dataset을 불러온다음 길이를 계산하여\
train : 0.8 / validation : 0.1 / test : 0.1 로 분리하여\
dataloader를 이용해 각각 불러와 train이후 validation loss를 구할 것이다.


### Dataset 구성

In [4]:
class CustomDataset(Dataset):
  def __init__(self,file_path):
    df=pd.read_csv(file_path)
    self.x=df.iloc[:,0].values
    self.y=df.iloc[:,1].values
    self.length=len(df)

  def __getitem__(self,index):
    x=torch.FloatTensor([self.x[index]*2,self.x[index]])
    y=torch.FloatTensor([self.y[index]])
    return x,y

  def __len__(self):
    return self.length

### Model 구성

In [8]:
class CustomModel(nn.Module):
  def __init__(self):
    super(CustomModel,self).__init__()
    self.layer=nn.Linear(2,1)

  def forward(self,x):
    x=self.layer(x)
    return x

### Dataset 불러오기 및 Train,Validation,Test 분류

In [6]:
dataset=CustomDataset("dataset.csv")
dataset_size=len(dataset)
train_size=int(dataset_size*0.8)  # 꼭 정수형(int)으로 바꿔줘야 한다!!!
validation_size=int(dataset_size*0.1)
test_size=dataset_size-train_size-validation_size

train_dataset,validation_dataset,test_dataset=random_split(dataset,[train_size,validation_size,test_size])

print(f"Train_dataset_size = {len(train_dataset)}")
print(f"Validation_dataset_size = {len(validation_dataset)}")
print(f"Test_dataset_size = {len(test_dataset)}")

Train_dataset_size = 160
Validation_dataset_size = 20
Test_dataset_size = 20


### 모델,Loss,Optimizer 계산에 GPU를 사용할 수 있음

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(),lr=0.0001)

### 학습